# BTC is non linear.

`BTC follows logarithmic regression`

`Volatility can be applied `

Even in machine learning (like your RL model), nonlinear techniques (such as LSTMs, reinforcement learning, and deep neural networks) are better suited for price prediction.

Conclusion
BTC price is highly nonlinear, best analyzed using logarithmic trends, power laws, and fractals rather than simple linear models. If you're working on a trading model, incorporating nonlinear functions and time-series methods would be more effective. 🚀

scaler = RobustScaler()
✅ Good for BTC's sudden crashes & pumps.
✅ Works well with models that are sensitive to outliers.

StandardScaler	Normally distributed data	❌ Not ideal
MinMaxScaler	Neural networks with sigmoid/tanh	⚠️ Maybe (not for BTC)
Log Scaling	Exponential price trends	✅ Best for BTC
Power Transform	Heavy-tailed distributions	✅ Good alternative
RobustScaler	Data with outliers	✅ Works well for BTC

# Model yorumları

Model 1-2-3 (activation = ('tanh')) Çok daha iyi predictionslar veriyor.


## Pip & Imports

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from google.colab import files
import pandas as pd
import datetime
import time
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import RobustScaler

## Data read

In [2]:
# Deneme datayı alıyoruz
prices = pd.read_csv("denemelik.csv")  # Placeholder for your dataset
prices.set_index('Timestamp', inplace=True)

In [3]:
# ✅ Scaling with RobustScaler (better for handling BTC's outliers)
scaler = RobustScaler()
scaled_prices = scaler.fit_transform(prices)

# ✅ Convert back to DataFrame
scaled_prices = pd.DataFrame(scaled_prices, columns=prices.columns, index=prices.index)

look_back = 240  # Use 240 past candles as input
future_step = 1

X, y = [], []
for i in range(len(scaled_prices) - look_back - future_step):
    X.append(scaled_prices.iloc[i : i + look_back].values)  # Collect 240 timesteps
    y.append(scaled_prices.iloc[i + look_back + future_step - 1]["Close"])  # Close at t+1

# Convert to NumPy arrays
X = np.array(X)  # Shape: (samples, 240, features)
y = np.array(y)  # Shape: (samples,)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# ✅ Get the datetime index for the test set
X_test_indices = scaled_prices.index[-len(X_test):]


In [4]:
# Checking the data shape
print(f" X_train shape = {X_train.shape}")  # (N, 1, 4) --> Correct shape
print(f"X_test shape = {X_test.shape}")  # (N, 1, 4) --> Correct shape
print(f" y_train shape = {y_train.shape}")  # (N, 1, 4) --> Correct shape
print(f" y_test shape = {y_test.shape}")  # (N, 1, 4) --> Correct shape

 X_train shape = (7807, 240, 9)
X_test shape = (1952, 240, 9)
 y_train shape = (7807,)
 y_test shape = (1952,)


## Fonks

In [5]:
def produce_predictions(model,scaled_prices):
    y_pred = model.predict(X_test)
    df_pre =  pd.DataFrame(columns=["timestamp","p_price", "r_price"])
    X_test_indices = scaled_prices.index[-len(X_test):]
    for i in range(len(y_pred)):
      y_pred_inv = scaler.inverse_transform(
        [[0, y_pred[i][0], 0, 0, 0, 0, 0, 0, 0]]
        )[0][1]
      y_test_inv = scaler.inverse_transform(
        [[0, y_test[i], 0, 0, 0, 0, 0, 0, 0]]
        )[0][1]
      new_row = pd.DataFrame([{"timestamp":X_test_indices[i], "p_price": y_pred_inv, "r_price": y_test_inv}])
      # Use pd.concat() to add the new row
      df_pre = pd.concat([df_pre, new_row], ignore_index=True)
    df_pre.to_csv(f"{model.name}_tahmin.csv")
    files.download(f"{model.name}_tahmin.csv")
    return df_pre

In [6]:
def plotting_predictions(df_pre, filename):
  # Convert timestamp to datetime (if not already done)
  df_pre['timestamp'] = pd.to_datetime(df_pre['timestamp'])
  # Create a line plot
  plt.figure(figsize=(14, 8))
  plt.plot( df_pre['timestamp'], df_pre['p_price'], label='Predicted Price (pre_price)', color='blue', marker='o', markersize=4)
  plt.plot(df_pre['timestamp'], df_pre['r_price'], label='Real Price (real_price)', color='red', marker='o', markersize=4)
  # Customize the axes and figure background
  ax = plt.gca()
  ax.set_facecolor('black')  # Set axes' background to black
  ax.tick_params(axis='both', colors='black')  # Set tick marks and labels to white
  ax.xaxis.label.set_color('white')  # Set x-axis label to white
  ax.yaxis.label.set_color('white')  # Set y-axis label to white
  ax.title.set_color('black')  # Set title color to white
  # Format the x-axis for minute-based grids
  ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=1))  # Grid every 5 minutes
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%M'))  # Format as HH:MM
  # Add labels and title
  plt.xlabel('Timestamp')
  plt.ylabel('Price')
  plt.title('Predicted vs Real Price Over Time')
  plt.legend()
  plt.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray')
  # Rotate x-axis labels for better readability
  plt.xticks(rotation=45)
  # Show the plot
  plt.tight_layout()
  plt.savefig(filename, format='png')  # Change the format if needed, e.g., 'jpg', 'svg'
  files.download(filename)  # This will prompt you to download the file
  plt.show()

# (1-100)- 2 LSTM 1 Dense layer. (activation ='tanh' + 1)

In [7]:
for i in range (1,100):
  model = Sequential([
      LSTM(i, return_sequences=True, input_shape=(240, 9), activation='tanh'),
      LSTM(i, return_sequences=False, activation = 'tanh'),
      Dense(1)
  ],name=f"model_{i}")
  print(model.name)
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse')
  # Early stopping callback
  #early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  print(f"{model.name} fitting is starting ----------------------------------->")
  model.fit(
      X_train, y_train,
      validation_split=0.1,
      epochs=20,
      batch_size=32,
      #callbacks=[early_stop]
  )
  eval = model.evaluate(X_test, y_test)
  print (f"Eval değerler = {eval} ----------------------------------->")
  y_pred = model.predict(X_test)
  # Doğru tahminleri bul
  y_pred = np.squeeze(y_pred)
  correct_predictions = y_pred == y_test
  count_values = pd.Series(correct_predictions).value_counts()
  if any(count_values.index == True):
      # Eğer True değeri varsa burası çalışır
      print("En az bir true değeri mevcut!---------------->")
      df_pre_6 = produce_predictions(model,scaled_prices)
      plotting_predictions(df_pre_6[0:30], f"{model.name}.png")
      model.save(f"{model.name}.h5")
      files.download(f"{model.name}.h5")
  else:
    print("True değer yok :( -------------------------> \n ----------------->Yeni Modele geçiliyor.")



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


model_1
model_1 fitting is starting ----------------------------------->
Epoch 1/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 41s 165ms/step - loss: 0.1873 - val_loss: 0.0043
Epoch 2/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 37s 166ms/step - loss: 0.0055 - val_loss: 0.0099
Epoch 3/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 35s 159ms/step - loss: 0.0025 - val_loss: 0.0111
Epoch 4/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 42s 165ms/step - loss: 0.0019 - val_loss: 0.0046
Epoch 5/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 42s 168ms/step - loss: 0.0014 - val_loss: 0.0017
Epoch 6/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 39s 158ms/step - loss: 0.0012 - val_loss: 0.0028
Epoch 7/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 43s 166ms/step - loss: 0.0011 - val_loss: 0.0014
Epoch 8/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 36s 165ms/step - loss: 0.0012 - val_loss: 0.0024
Epoch 9/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 39s 157ms/step - loss: 0.0012 - val_loss: 0.0010
Epoch 10/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 43s 165ms/step - loss: 9.6458e-04 - val_loss: 0.0029
Epoch 11/20
220/220 ━━━━━━━━━━━━

KeyboardInterrupt: 

# 15 m pred


In [ ]:
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    # Separate gains and losses
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    # Calculate rolling averages
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    # Compute Relative Strength (RS)
    rs = avg_gain / avg_loss
    # Compute RSI
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [ ]:
import yfinance as yf
dfr = yf.download(tickers="BTC-USD", period="3d", interval="15m")
dfr.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
desired_order = ["Open", "Close",	"High",	"Low", "Volume"]
dfr = dfr[desired_order]
dfr.index.rename("Timestamp", inplace=True)
df1 = dfr

[*********************100%***********************]  1 of 1 completed


In [ ]:
df1["RSI30"] = calculate_rsi(df1, window=30)
df1["RSI15"] = calculate_rsi(df1, window=15)

In [ ]:
df = df1.iloc[-240:]

In [ ]:
df

,Open,Close,High,Low,Volume,RSI30,RSI15
Timestamp,,,,,,,
2025-02-06 09:45:00,98682.835938,98539.265625,98734.859375,98539.265625,0,65.343833,65.621747
2025-02-06 10:00:00,98870.742188,98674.023438,98870.742188,98674.023438,982679552,69.125581,65.953996
2025-02-06 10:15:00,98986.101562,98886.554688,98986.101562,98809.398438,80113664,70.097044,67.441508
2025-02-06 10:30:00,98784.375000,98926.890625,98937.625000,98710.085938,147075072,70.020159,63.950469
2025-02-06 10:45:00,98959.664062,98904.960938,99041.937500,98877.578125,609267712,72.896852,62.133899
...,...,...,...,...,...,...,...
2025-02-08 20:30:00,96438.453125,96491.414062,96491.414062,96438.453125,979644416,57.401731,71.704287
2025-02-08 20:45:00,96478.312500,96435.960938,96494.203125,96391.265625,1513549824,54.434154,63.694883
2025-02-08 21:00:00,96440.343750,96464.703125,96567.148438,96433.804688,52631552,54.671796,60.815385


In [ ]:
df.shape

(240, 7)

In [ ]:
# Preprocess and normalize
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

X_df = df_scaled
y_df = df_scaled['Close']
n_df_X = np.array(X_df)
n_df_y = np.array(y_df)  # Shape: (samples,)
new_indices = df_scaled.index[-len(df_scaled):]

In [ ]:
n_df_X.shape

(240, 7)

In [ ]:
n_df_X = n_df_X.reshape(1, 240, 7)

In [ ]:
n_df_X.shape

(1, 240, 7)

In [ ]:
y_pred = a.predict(n_df_X)

y_pred_inv = scaler.inverse_transform(
[[0, y_pred[0][0], 0, 0, 0, 0, 0]]
)[0][1]

print(y_pred_inv)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step
96485.43163468502
